In [1]:
import requests
import json
import numpy as np
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("tp-sentiment'analysis").getOrCreate()

In [2]:
r = requests.get(url='https://raw.githubusercontent.com/lucaspando/yelp-reviews/master/review/yelp_academic_dataset_review-001-001.json')

jsonStr = r.content.decode("utf-8")
print(jsonStr)

linelist = jsonStr.splitlines()
count = len(linelist)

{"review_id":"x7mDIiDB3jEiPGPHOmDzyw","user_id":"msQe1u7Z_XuqjGoqhB0J5g","business_id":"iCQpiavjjPzJ5_3gPD5Ebg","stars":2,"date":"2011-02-25","text":"The pizza was okay. Not the best I've had. I prefer Biaggio's on Flamingo \/ Fort Apache. The chef there can make a MUCH better NY style pizza. The pizzeria @ Cosmo was over priced for the quality and lack of personality in the food. Biaggio's is a much better pick if youre going for italian - family owned, home made recipes, people that actually CARE if you like their food. You dont get that at a pizzeria in a casino. I dont care what you say...","useful":0,"funny":0,"cool":0}
{"review_id":"dDl8zu1vWPdKGihJrwQbpw","user_id":"msQe1u7Z_XuqjGoqhB0J5g","business_id":"pomGBqfbxcqPv14c3XH-ZQ","stars":5,"date":"2012-11-13","text":"I love this place! My fiance And I go here atleast once a week. The portions are huge! Food is amazing. I love their carne asada. They have great lunch specials... Leticia is super nice and cares about what you think of her restaurant. You have to try their cheese enchiladas too the sauce is different And amazing!!!","useful":0,"funny":0,"cool":0}
{"review_id":"LZp4UX5zK3e-c5ZGSeo3kA","user_id":"msQe1u7Z_XuqjGoqhB0J5g","business_id":"jtQARsP6P-LbkyjbO1qNGg","stars":1,"date":"2014-10-23","text":"Terrible. Dry corn bread. Rib tips were all fat and mushy and had no flavor. If you want bbq in this neighborhood go to john mulls roadkill grill. Trust me.","useful":3,"funny":1,"cool":1}
{"review_id":"Er4NBWCmCD4nM8_p1GRdow","user_id":"msQe1u7Z_XuqjGoqhB0J5g","business_id":"elqbBhBfElMNSrjFqW3now","stars":2,"date":"2011-02-25","text":"Back in 2005-2007 this place was my FAVORITE thai place EVER. I'd go here ALLLLL the time. I never had any complaints. Once they started to get more known and got busy, their service started to suck and their portion sizes got cut in half. I have a huge problem with paying MORE for way less food. The last time I went there I had the Pork Pad se Ew and it tasted good, but I finished my plate and was still hungry. I used to know the manager here and she would greet me with a \"Hello Melissa, nice to see you again, diet coke & pad thai or pad se ew?\" Now a days, I know she still knows me but she disregards my presence. Also, I had asked her what was up with the new portion sizes and she had no answer for me. Great food but not worth the money. I havent been back in over a year because I refuse to pay $10-15 for dinner and still be hungry after. Sorry PinKaow, you are not what you used to be!!","useful":2,"funny":0,"cool":0}
{"review_id":"jsDu6QEJHbwP2Blom1PLCA","user_id":"msQe1u7Z_XuqjGoqhB0J5g","business_id":"Ums3gaP2qM3W1XcA5r6SsQ","stars":5,"date":"2014-09-05","text":"Delicious healthy food. The steak is amazing. Fish and pork are awesome too. Service is above and beyond. Not a bad thing to say about this place. Worth every penny!","useful":0,"funny":0,"cool":0}
{"review_id":"pfavA0hr3nyqO61oupj-lA","user_id":"msQe1u7Z_XuqjGoqhB0J5g","business_id":"vgfcTvK81oD4r50NMjU2Ag","stars":1,"date":"2011-02-25","text":"This place sucks. The customer service is horrible. They dont serve food unless you order a pizza from a neighboring restaurant. Who does that? They dont control their crowd. Many times I've gone I've seen fights. The bartenders suck - I've almost got in a fight with one because she was a complete bitch. Refused to serve me a drink because she was \"busy\" celebrating her friends birthday BEHIND THE BAR. This place is ridiculous. I will NEVER go there again.. EVER.","useful":2,"funny":0,"cool":0}
{"review_id":"brokEno2n7s4vrwmmUdr9w","user_id":"msQe1u7Z_XuqjGoqhB0J5g","business_id":"AxeQEz3-s9_1TyIo-G7UQw","stars":5,"date":"2011-10-10","text":"If you like Thai food, you have to try the original thai bbq. Their pad se ew is to DIE for. Their thai egg rolls are delicious. Basil beef will not let you down (its not on the menu anymore, you have to ask for it!) \n\nYes, the building is not as fancy as some other places. Yes, i've batte

In [3]:
def prepareData(line):
    line['text'] = line['text'].replace("\n", "")
    line['text'] = line['text'].replace("\\", "")
    line['text'] = line['text'].replace("\'", " ")
    line['text'] = line['text'].replace("/", "")
    line['text'] = line['text'].lower()
    line['good'] = 1 if line['stars'] >= 4 else 0
    splittedDate = line['date'].split("-")
    line['year'] = int(splittedDate[0])
    line['month'] = int(splittedDate[1])
    line['day'] = int(splittedDate[2])
    return line

In [4]:
data = []

for i in range(count-1):
  json_review = json.loads(linelist[i])
  json_review = prepareData(json_review)
  data.append(json_review)

print("Example review:", data[7])

Example review: {'funny': 1, 'day': 18, 'business_id': 'zdE82PiD6wquvjYLyhOJNA', 'date': '2012-04-18', 'useful': 0, 'text': 'amazing!!! i was referred here by a friend and at first thought "korean and mexican?? weird... i dont know"... but my husband and i were in the area and thought why not, lets try it... after my friend had nothing but great things to say about this place. we tried the chimichanga and it was absolutely amazing.... we actually did 12 chicken 12 beef (have you ever been to a place that will make a burrito half and half? i havent!) the meat has fantastic flavor. i m not a beef girl but i d order a whole plate of their beef - its that good. the chimichanga was huge enough for both of us to share it (and my husband also had a beef taco which also was delicious) and we were full and it was $11 total with drinks. you cant beat that.the service was great. the server was so nice and attentive- you could tell she was the owner. the only thing i d change is their chips they serve. they were burnt, too thick and kind of bland but their salsa is great. and lets face it, i m fat so i we ate two baskets anyway so they cant be that bad. lolall in all, food was delicious, a lot of food for cheap, great prices, great service great flavor. hell even my burps taste delicious. bahaha!', 'review_id': 'kUZWBVZvhWuC8TWUg5AYyA', 'year': 2012, 'month': 4, 'user_id': 'msQe1u7Z_XuqjGoqhB0J5g', 'good': 1, 'stars': 5, 'cool': 0}

In [5]:
json_rdd = sc.parallelize(data)
df = spark.read.json(json_rdd)
df.printSchema()

root
-- _corrupt_record: string (nullable = true)
-- business_id: string (nullable = true)
-- cool: long (nullable = true)
-- date: string (nullable = true)
-- day: long (nullable = true)
-- funny: long (nullable = true)
-- good: long (nullable = true)
-- month: long (nullable = true)
-- review_id: string (nullable = true)
-- stars: long (nullable = true)
-- text: string (nullable = true)
-- useful: long (nullable = true)
-- user_id: string (nullable = true)
-- year: long (nullable = true)

In [6]:
display(df)

_corrupt_record,business_id,cool,date,day,funny,good,month,review_id,stars,text,useful,user_id,year
null,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,25,0,0,2,x7mDIiDB3jEiPGPHOmDzyw,2,"the pizza was okay. not the best i ve had. i prefer biaggio s on flamingo fort apache. the chef there can make a much better ny style pizza. the pizzeria @ cosmo was over priced for the quality and lack of personality in the food. biaggio s is a much better pick if youre going for italian - family owned, home made recipes, people that actually care if you like their food. you dont get that at a pizzeria in a casino. i dont care what you say...",0,msQe1u7Z_XuqjGoqhB0J5g,2011
null,pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,13,0,1,11,dDl8zu1vWPdKGihJrwQbpw,5,i love this place! my fiance and i go here atleast once a week. the portions are huge! food is amazing. i love their carne asada. they have great lunch specials... leticia is super nice and cares about what you think of her restaurant. you have to try their cheese enchiladas too the sauce is different and amazing!!!,0,msQe1u7Z_XuqjGoqhB0J5g,2012
null,jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,23,1,0,10,LZp4UX5zK3e-c5ZGSeo3kA,1,terrible. dry corn bread. rib tips were all fat and mushy and had no flavor. if you want bbq in this neighborhood go to john mulls roadkill grill. trust me.,3,msQe1u7Z_XuqjGoqhB0J5g,2014
null,elqbBhBfElMNSrjFqW3now,0,2011-02-25,25,0,0,2,Er4NBWCmCD4nM8_p1GRdow,2,"back in 2005-2007 this place was my favorite thai place ever. i d go here alllll the time. i never had any complaints. once they started to get more known and got busy, their service started to suck and their portion sizes got cut in half. i have a huge problem with paying more for way less food. the last time i went there i had the pork pad se ew and it tasted good, but i finished my plate and was still hungry. i used to know the manager here and she would greet me with a ""hello melissa, nice to see you again, diet coke & pad thai or pad se ew?"" now a days, i know she still knows me but she disregards my presence. also, i had asked her what was up with the new portion sizes and she had no answer for me. great food but not worth the money. i havent been back in over a year because i refuse to pay $10-15 for dinner and still be hungry after. sorry pinkaow, you are not what you used to be!!",2,msQe1u7Z_XuqjGoqhB0J5g,2011
null,Ums3gaP2qM3W1XcA5r6SsQ,0,2014-09-05,5,0,1,9,jsDu6QEJHbwP2Blom1PLCA,5,delicious healthy food. the steak is amazing. fish and pork are awesome too. service is above and beyond. not a bad thing to say about this place. worth every penny!,0,msQe1u7Z_XuqjGoqhB0J5g,2014
null,vgfcTvK81oD4r50NMjU2Ag,0,2011-02-25,25,0,0,2,pfavA0hr3nyqO61oupj-lA,1,"this place sucks. the customer service is horrible. they dont serve food unless you order a pizza from a neighboring restaurant. who does that? they dont control their crowd. many times i ve gone i ve seen fights. the bartenders suck - i ve almost got in a fight with one because she was a complete bitch. refused to serve me a drink because she was ""busy"" celebrating her friends birthday behind the bar. this place is ridiculous. i will never go there again.. ever.",2,msQe1u7Z_XuqjGoqhB0J5g,2011
null,AxeQEz3-s9_1TyIo-G7UQw,0,2011-10-10,10,0,1,10,brokEno2n7s4vrwmmUdr9w,5,"if you like thai food, you have to try the original thai bbq. their pad se ew is to die for. their thai egg rolls are delicious. basil beef will not let you down (its not on the menu anymore, you have to ask for it!) yes, the building is not as fancy as some other places. yes, i ve batted a fly off my plate more than once. yes, i do not go to the bathroom their because i dont even want to know what it looks like... but.. the thai food is the best in town. the service rocks. and you can get a $25 gift cert. on restaurant.com for $2. can you beat that? i think not.this is the only place my husband and i go for anniversarys, date nights, birthdays.. anything!! i recommend it to everyone i know. if you know good thai 

In [7]:
df = df.drop('_corrupt_record', 'cool', 'funny', 'review_id', 'user_id')

In [8]:
countBeforeCleaning = df.count()
print(countBeforeCleaning)
df.describe().show()

28926
+-------+--------------------+----------+------------------+-------------------+------------------+------------------+--------------------+------------------+------------------+
summary| business_id| date| day| good| month| stars| text| useful| year|
+-------+--------------------+----------+------------------+-------------------+------------------+------------------+--------------------+------------------+------------------+
 count| 28903| 28903| 28903| 28903| 28903| 28903| 28903| 28903| 28903|
 mean| null| null|15.792236100058817| 0.647026260249801| 6.555374874580493|3.6611078434764557| null|1.0375047572916307|2014.2768224751755|
 stddev| null| null| 8.802623086241061|0.47790289890908366|3.3460131257307664|1.5533502751330637| null| 3.853536753932989|2.1696612759731684|
 min|--9e1ONYQuAa-CB_R...|2006-02-16| 1| 0| 1| 1| a pretentious hy...| 0| 2006|
 max|zzwhN7x37nyjP0ZM8...|2018-01-05| 31| 1| 12| 5|~~~~~brilliant! ~...| 536| 2018|
+-------+--------------------+----------+------------------+-------------------+------------------+------------------+--------------------+------------------+------------------+

In [9]:
from pyspark.sql.functions import isnan, when, count, col

cols = df.columns 
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in cols]).show()

+-----------+----+---+----+-----+-----+----+------+----+
business_id|date|day|good|month|stars|text|useful|year|
+-----------+----+---+----+-----+-----+----+------+----+
 23| 23| 23| 23| 23| 23| 23| 23| 23|
+-----------+----+---+----+-----+-----+----+------+----+

In [10]:
df = df.dropna()
print("Cantidad de elementos antes de la limpieza: ", countBeforeCleaning)
print("Cantidad de elementos luego de la limpieza: ", df.count())
print("Se realizo una limpieza de ", countBeforeCleaning - df.count(), " registros") 

Cantidad de elementos antes de la limpieza: 28926
Cantidad de elementos luego de la limpieza: 28903
Se realizo una limpieza de 23 registros

In [11]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in cols]).show()

+-----------+----+---+----+-----+-----+----+------+----+
business_id|date|day|good|month|stars|text|useful|year|
+-----------+----+---+----+-----+-----+----+------+----+
 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-----------+----+---+----+-----+-----+----+------+----+

In [12]:
df.createOrReplaceTempView("reviews")

In [13]:
%sql
SELECT stars, COUNT(*)
FROM reviews
GROUP By stars
ORDER BY 2 DESC

stars,count(1)
5,13389
4,5312
1,5260
3,2480
2,2462


In [14]:
%sql
SELECT year, COUNT(*)
FROM reviews
GROUP By year
ORDER By 2 DESC

year,count(1)
2016,5598
2015,5539
2014,4492
2017,4287
2013,3162
2012,2064
2011,1789
2010,1120
2009,464
2008,273


In [15]:
%sql
SELECT year, stars, COUNT(*)
FROM reviews
GROUP By year, stars
ORDER BY year

year,stars,count(1)
2006,4,4
2006,5,7
2006,1,1
2006,3,2
2007,5,18
2007,4,33
2007,2,5
2007,3,11
2007,1,5
2008,2,35


In [16]:
%sql
SELECT year, stars, COUNT(*) as StarsQuantity
FROM reviews
GROUP By year, stars
ORDER By year

year,stars,StarsQuantity
2006,4,4
2006,5,7
2006,1,1
2006,3,2
2007,5,18
2007,2,5
2007,4,33
2007,1,5
2007,3,11
2008,4,89


In [17]:
# Inicio del analisis de sentimiento, donde vamos a intentar realizar un modelo que realice una prediccion 
# si la review de un negocio es buena ('good': >= 4 stars) a partir del texto escrito.

In [18]:
(train_set, val_set, test_set) = df.randomSplit([0.8, 0.1, 0.1], seed = 17)
print("Cantidad training: ", train_set.count())
print("Cantidad validation: ", val_set.count())
print("Cantidad test: ", test_set.count())

Cantidad training: 23176
Cantidad validation: 2831
Cantidad test: 2896

In [19]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml import Pipeline

tokenizer = Tokenizer(inputCol="text", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
hashtf = HashingTF(numFeatures=2**16, inputCol="filtered", outputCol='tf')
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
label_stringIdx = StringIndexer(inputCol = "good", outputCol = "label")
pipeline = Pipeline(stages=[tokenizer, remover, hashtf, idf, label_stringIdx])

pipelineFit = pipeline.fit(train_set)
train_df = pipelineFit.transform(train_set)
val_df = pipelineFit.transform(val_set)
train_df.show(20)

+--------------------+----------+---+----+-----+-----+--------------------+------+----+--------------------+--------------------+--------------------+--------------------+-----+
 business_id| date|day|good|month|stars| text|useful|year| words| filtered| tf| features|label|
+--------------------+----------+---+----+-----+-----+--------------------+------+----+--------------------+--------------------+--------------------+--------------------+-----+
--cZ6Hhc9F7VkKXxH...|2012-12-19| 19| 1| 12| 5|our family of fiv...| 1|2012|[our, family, of,...|[family, five, go...|(65536,[14,556,29...|(65536,[14,556,29...| 0.0|
-050d_XIor1NpCuWk...|2009-08-29| 29| 1| 8| 5|one of the best b...| 0|2009|[one, of, the, be...|[one, best, break...|(65536,[14,3191,6...|(65536,[14,3191,6...| 0.0|
-0qht1roIqleKiQkB...|2017-05-10| 10| 0| 5| 1|i went to the caf...| 0|2017|[i, went, to, the...|[went, cafebar, p...|(65536,[981,2346,...|(65536,[981,2346,...| 1.0|
-2ToCaDFpTNmmg3QF...|2014-11-08| 8| 0| 11| 1|wish i had read t...| 0|2014|[wish, i, had, re...|[wish, read, revi...|(65536,[3921,4894...|(65536,[3921,4894...| 1.0|
-3zffZUHoY8bQjGfP...|2011-07-13| 13| 1| 7| 5|while it s been n...| 0|2011|[while, it, s, be...|[nearly, year, si...|(65536,[389,1133,...|(65536,[389,1133,...| 0.0|
-A5jntJgFglQ6zwAm...|2013-08-17| 17| 1| 8| 5|i stayed at the b...| 1|2013|[i, stayed, at, t...|[stayed, baymont,...|(65536,[2955,4318...|(65536,[2955,4318...| 0.0|
-AD5PiuJHgdUcAK-V...|2012-03-13| 13| 1| 3| 4|have been to seve...| 0|2012|[have, been, to, ...|[several, "best, ...|(65536,[3795,2001...|(65536,[3795,2001...| 0.0|
-BOdPJnlQlLRv2aZ0...|2012-12-29| 29| 1| 12| 5|i cannot say enou...| 4|2012|[i, cannot, say, ...|[say, enough, chu...|(65536,[14,1143,1...|(65536,[14,1143,1...| 0.0|
-BSf1LtnOrtCWAzmu...|2014-02-19| 19| 1| 2| 4|i have recently d...| 2|2014|[i, have, recentl...|[recently, discov...|(65536,[329,1217,...|(65536,[329,1217,...| 0.0|
-C8sSrFqaCxp51pyo...|2016-12-08| 8| 1| 12| 5|just had the kent...| 0|2016|[just, had, the, ...|[kentucky, fried,...|(65536,[2406,3288...|(65536,[2406,3288...| 0.0|
-CfFjcCcGGDM9MVH_...|2015-08-11| 11| 1| 8| 5|excellent mediter...| 0|2015|[excellent, medit...|[excellent, medit...|(65536,[30,3191,3...|(65536,[30,3191,3...| 0.0|
-G7MPSNBpxRJmtrJx...|2015-02-27| 27| 1| 2| 5|very authentic pu...| 0|2015|[very, authentic,...|[authentic, pupus...|(65536,[111,166,1...|(65536,[111,166,1...| 0.0|
-GY2fx-8udXPY8qn2...|2016-08-30| 30| 1| 8| 5|just opened a new...| 1|2016|[just, opened, a,...|[opened, new, acc...|(65536,[329,3149,...|(65536,[329,3149,...| 0.0|
-HgbBv9Yjqv735F9f...|2012-06-29| 29| 1| 6| 4|the times i have ...| 0|2012|[the, times, i, h...|[times, impressed...|(65536,[308,3795,...|(65536,[308,3795,...| 0.0|
-Ht7HiGBox8lS1Y8I...|2012-04-17| 17| 1| 4| 5|in town on busine...| 0|2012|[in, town, on, bu...|[town, business, ...|(65536,[1832,5456...|(65536,[1832,5456...| 0.0|
-J6FVdY9pSgAdFmma...|2016-03-06| 6| 0| 3| 1|not authentic at ...| 0|2016|[not, authentic, ...|[authentic, all.....|(65536,[254,1198,...|(65536,[254,1198,...| 1.0|
-L88B-PrtzwujhLuY...|2015-07-18| 18| 0| 7| 1|edit: 83015 - jus...| 1|2015|[edit:, 83015, -,...|[edit:, 83015, -,...|(65536,[389,402,1...|(65536,[389,402,1...| 1.0|
-LRlx2j9_LB3evsRR...|2017-10-07| 7| 1| 10| 5|the vet took the ...| 0|2017|[the, vet, took, ...|[vet, took, time,...|(65536,[1198,1706...|(65536,[1198,1706...| 0.0|
-MKWJZnMjSit406AU...|2015-01-03| 3| 0| 1| 1|drove 37 miles on...| 0|2015|[drove, 37, miles...|[drove, 37, miles...|(65536,[8612,1861...|(65536,[8612,1861...| 1.0|
-Ma5JqzuI_3xIAtpN...|2017-05-24| 24| 1| 5| 5|this neighborhood...| 0|2017|[this, neighborho...|[neighborhood, th...|(65536,[7284,1099...|(65536,[7284,1099...| 0.0|
+--------------------+----------+---+----+-----+-----+--------------------+------+----+--------------------+--------------------+--------------------+--------------------+-----+
only showing top 20 rows

In [20]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=80)
lrModel = lr.fit(train_df)
predictions = lrModel.transform(val_df)

#predictions.show()

from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

Out[ 52 ]: 0.8979050355002826

In [21]:
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(val_set.count())
accuracy

Out[ 53 ]: 0.8396326386435888

In [22]:
predictions.select('good', 'prediction', 'stars').show()

+----+----------+-----+
good|prediction|stars|
+----+----------+-----+
 0| 0.0| 2|
 1| 0.0| 5|
 1| 0.0| 5|
 1| 0.0| 5|
 1| 0.0| 4|
 1| 0.0| 4|
 1| 0.0| 4|
 0| 0.0| 1|
 1| 0.0| 5|
 1| 0.0| 5|
 1| 1.0| 5|
 1| 0.0| 5|
 1| 0.0| 5|
 1| 0.0| 5|
 1| 1.0| 5|
 0| 0.0| 3|
 1| 0.0| 5|
 1| 0.0| 4|
 0| 0.0| 3|
 1| 0.0| 5|
+----+----------+-----+
only showing top 20 rows

In [23]:
#Conclusión, obtenemos un accuracy del 84% aproximadamente, esto se podría mejorar de varias formas, por ejemplo utilizando algun otro metodo de machine learning como por ejemplo SVM o una red neurnal, hacer una comparación de las performances de cada una y quedarse con la más adecuadad. También podríamos intentar purgar los datos con otros mecanismos como por ejemplo haciendo stemming de las palabras (Obtieniendo solamente la raiz de las mismas, probacando que haya similitud entre las palabras que pertenecen a una misma familia). O también se podría cambiar alguno de los parametros de el método utilizado y ver si aumenta la performance de este modelo.

In [24]:
linelist_business = []

for i in range(1,7):
  r = requests.get(url='https://raw.githubusercontent.com/lucaspando/yelp-reviews/master/business/yelp_academic_dataset_business.json.00' + str(i))
  jsonStr = r.content.decode("utf-8")
  linelist_business = np.concatenate((linelist_business,jsonStr.splitlines()),axis=0)


In [25]:
data_business = []

for i in range(len(linelist_business)-1):
  data_business.append(json.loads(linelist_business[i]))

In [26]:
json_rdd = sc.parallelize(data_business)
dfBusiness = spark.read.json(json_rdd)
dfBusiness.printSchema()

root
-- _corrupt_record: string (nullable = true)
-- address: string (nullable = true)
-- attributes: struct (nullable = true)
 |-- AcceptsInsurance: string (nullable = true)
 |-- AgesAllowed: string (nullable = true)
 |-- Alcohol: string (nullable = true)
 |-- Ambience: string (nullable = true)
 |-- BYOB: string (nullable = true)
 |-- BYOBCorkage: string (nullable = true)
 |-- BestNights: string (nullable = true)
 |-- BikeParking: string (nullable = true)
 |-- BusinessAcceptsBitcoin: string (nullable = true)
 |-- BusinessAcceptsCreditCards: string (nullable = true)
 |-- BusinessParking: string (nullable = true)
 |-- ByAppointmentOnly: string (nullable = true)
 |-- Caters: string (nullable = true)
 |-- CoatCheck: string (nullable = true)
 |-- Corkage: string (nullable = true)
 |-- DietaryRestrictions: string (nullable = true)
 |-- DogsAllowed: string (nullable = true)
 |-- DriveThru: string (nullable = true)
 |-- GoodForDancing: string (nullable = true)
 |-- GoodForKids: string (nullable = true)
 |-- GoodForMeal: string (nullable = true)
 |-- HairSpecializesIn: string (nullable = true)
 |-- HappyHour: string (nullable = true)
 |-- HasTV: string (nullable = true)
 |-- Music: string (nullable = true)
 |-- NoiseLevel: string (nullable = true)
 |-- Open24Hours: string (nullable = true)
 |-- OutdoorSeating: string (nullable = true)
 |-- RestaurantsAttire: string (nullable = true)
 |-- RestaurantsCounterService: string (nullable = true)
 |-- RestaurantsDelivery: string (nullable = true)
 |-- RestaurantsGoodForGroups: string (nullable = true)
 |-- RestaurantsPriceRange2: string (nullable = true)
 |-- RestaurantsReservations: string (nullable = true)
 |-- RestaurantsTableService: string (nullable = true)
 |-- RestaurantsTakeOut: string (nullable = true)
 |-- Smoking: string (nullable = true)
 |-- WheelchairAccessible: string (nullable = true)
 |-- WiFi: string (nullable = true)
-- business_id: string (nullable = true)
-- categories: string (nullable = true)
-- city: string (nullable = true)
-- hours: struct (nullable = true)
 |-- Friday: string (nullable = true)
 |-- Monday: string (nullable = true)
 |-- Saturday: string (nullable = true)
 |-- Sunday: string (nullable = true)
 |-- Thursday: string (nullable = true)
 |-- Tuesday: string (nullable = true)
 |-- Wednesday: string (nullable = true)
-- is_open: long (nullable = true)
-- latitude: double (nullable = true)
-- longitude: double (nullable = true)
-- name: string (nullable = true)
-- neighborhood: string (nullable = true)
-- postal_code: string (nullable = true)
-- review_count: long (nullable = true)
-- stars: double (nullable = true)
-- state: string (nullable = true)

In [27]:
dfComplete = df.join(dfBusiness, df.business_id == dfBusiness.business_id, "left_outer").select(df.stars, df.business_id, 'text', 'year', 'month', 'day', 'useful', 'city', 'state', 'latitude', 'longitude', 'categories', 'review_count')

In [28]:
dfComplete.createOrReplaceTempView("reviewsReloaded")

In [29]:
%sql
SELECT city, stars, COUNT(*) as StarsQuantity
FROM reviewsReloaded
WHERE isNotNull(city)
GROUP By city, stars
ORDER By 3 Desc

city,stars,StarsQuantity
Las Vegas,5,2701
Phoenix,5,1142
Las Vegas,1,870
Las Vegas,4,851
Toronto,5,775
Scottsdale,5,764
Toronto,4,636
Charlotte,5,434
Las Vegas,3,396
Las Vegas,2,391


In [30]:
%sql
SELECT state, COUNT(*) as ReviewQuantity
FROM reviewsReloaded
WHERE isNotNull(state) AND state NOT IN ('ON', 'QC', 'AB', 'HE', 'NYK', 'BY', 'XGM', 'ST') /* Remove no US States */
GROUP By state
ORDER By 2 Desc

state,ReviewQuantity
AZ,6678
NV,5888
NC,1404
PA,1034
OH,1014
WI,584
IL,181
SC,55
IN,4
OR,2


In [31]:
%sql
SELECT state, AVG(stars) as AvgStars, Count(*) as ReviewQuantity
FROM reviewsReloaded
WHERE isNotNull(state)
GROUP By state
ORDER By 3 Desc

state,AvgStars,ReviewQuantity
AZ,3.7705899970050916,6678
NV,3.786514945652174,5888
ON,3.499056010069226,3178
NC,3.5712250712250713,1404
PA,3.6518375241779495,1034
OH,3.7337278106508878,1014
QC,3.7990506329113924,632
WI,3.714041095890411,584
AB,3.6579406631762654,573
IL,3.701657458563536,181


In [32]:
spark.stop()